# Predict Customer Churn Using Logistic Regression & PySpark

Logistic Regresion on a maketing agency's customer database to predict future customer churn on seen and unforeseen data.
The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company

In [0]:
# Start a new Spark session
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('logistic').getOrCreate()

In [0]:
# Read the dataset from the file
data=spark.read.csv('dbfs:/FileStore/shared_uploads/hrishagni95@gmail.com/customer_churn.csv',inferSchema=True,header=True)

In [0]:
# Check out the columns in the dataframe
data.columns

Out[3]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
# Sneak peek on the data that we are dealing with
data.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
 Jessica Williams|48.0| 10356.02| 0| 5.12| 8.0|2009-03-03 23:13:37|6187 Olson Mounta...| Kelly-Warren| 1|
 Eric Butler|44.0| 11331.58| 1| 5.23| 11.0|2016-12-05 03:35:43|4846 Savannah Roa...| Reynolds-Sheppard| 1|
 Zachary Walsh|32.0| 9885.12| 1| 6.92| 9.0|2006-03-09 14:50:20|25271 Roy Express...| Singh-Cole| 1|
 Ashlee Carr|43.0| 14062.6| 1| 5.46| 11.0|2011-09-29 05:47:23|3725 Caroline Str...| Lopez PLC| 1|
 Jennifer Lynch|40.0| 8066.94| 1| 7.11| 11.0|2006-03-28 15:42:45|363 Sandra Lodge ...| Reed-Martinez| 1|
 Paula Harris|30.0| 11575.37| 1| 5.22| 8.0|2016-11-13 13:13:01|Unit 8120 Box 916...|Briggs, Lamb and ...| 1|
 Bruce Phillips|45.0| 8771.02| 1| 6.64| 11.0|2015-05-28 12:14:03|Unit 1895 Box 094...| Figueroa-Maynard| 1|
 Craig Garner|45.0| 8988.67| 1| 4.84| 11.0|2011-02-16 08:10:47|897 Kelley Overpa...| Abbott-Thompson| 1|
 Nicole Olson|40.0| 8283.32| 1| 5.1| 13.0|2012-11-22 05:35:03|11488 Weaver Cape...|Smith, Kim and Ma...| 1|
 Harold Griffin|41.0| 6569.87| 1| 4.3| 11.0|2015-03-28 02:13:44|1774 Peter Row Ap...|Snyder, Lee and M...| 1|
 James Wright|38.0| 10494.82| 1| 6.81| 12.0|2015-07-22 08:38:40|45408 David Path ...| Sanders-Pierce| 1|
 Doris Wilkins|45.0| 8213.41| 1| 7.35| 11.0|2006-09-03 06:13:55|28216 Wright Moun...|Andrews, Adams an...| 1|
Katherine Carpenter|43.0| 11226.88| 0| 8.08| 12.0|2006-10-22 04:42:38|Unit 4948 Box 481...|Morgan, Phillips ...| 1|
 Lindsay Martin|53.0| 5515.09| 0| 6.85| 8.0|2015-10-07 00:27:10|69203 Crosby Divi...| Villanueva LLC| 1|
 Kathy Curry|46.0| 8046.4| 1| 5.69| 8.0|2014-11-06 23:47:14|9569 Caldwell Cre...|Berry, Orr and Ca...| 1|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [0]:
# Select only the below columns to consider for the regression and drop all the rows with null values in them
df=data.select('Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Churn').na.drop()

In [0]:
#Sneak peek of the data after filtering the original dataset
df.show()

+----+--------------+---------------+-----+---------+-----+
 Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+----+--------------+---------------+-----+---------+-----+
42.0| 11066.8| 0| 7.22| 8.0| 1|
41.0| 11916.22| 0| 6.5| 11.0| 1|
38.0| 12884.75| 0| 6.67| 12.0| 1|
42.0| 8010.76| 0| 6.71| 10.0| 1|
37.0| 9191.58| 0| 5.56| 9.0| 1|
48.0| 10356.02| 0| 5.12| 8.0| 1|
44.0| 11331.58| 1| 5.23| 11.0| 1|
32.0| 9885.12| 1| 6.92| 9.0| 1|
43.0| 14062.6| 1| 5.46| 11.0| 1|
40.0| 8066.94| 1| 7.11| 11.0| 1|
30.0| 11575.37| 1| 5.22| 8.0| 1|
45.0| 8771.02| 1| 6.64| 11.0| 1|
45.0| 8988.67| 1| 4.84| 11.0| 1|
40.0| 8283.32| 1| 5.1| 13.0| 1|
41.0| 6569.87| 1| 4.3| 11.0| 1|
38.0| 10494.82| 1| 6.81| 12.0| 1|
45.0| 8213.41| 1| 7.35| 11.0| 1|
43.0| 11226.88| 0| 8.08| 12.0| 1|
53.0| 5515.09| 0| 6.85| 8.0| 1|
46.0| 8046.4| 1| 5.69| 8.0| 1|
+----+--------------+---------------+-----+---------+-----+
only showing top 20 rows

In [0]:
# Import VectorAssembler and LogisticRegression from pyspark
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

In [0]:
# Use the below fields to create the 'features' field needed for regression
assembler=VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [0]:
# Create an instance of Logistic Regression
logistic_model=LogisticRegression(featuresCol='features',labelCol='Churn')

In [0]:
# Import the pipeline function from pyspark
from pyspark.ml.pipeline import Pipeline

In [0]:
# Start the pipeline and describe the stages
pipeline=Pipeline(stages=[assembler,logistic_model])

In [0]:
# Split the filtered dataset using 70/30 split and create training and testing data
train_data,test_data=df.randomSplit([0.7,0.3])

In [0]:
# Fit the pipeline with the training data
fit_model=pipeline.fit(train_data)

In [0]:
# Use the fitted model to transform the testing data set
res=fit_model.transform(test_data)

In [0]:
# Import BinaryClassificationEvaluator from pyspark and evaluate the results that we got from transforming the testing data set
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
# Create an instance of BinaryClassificationEvaluator
evaluation=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

In [0]:
# Check the AUROC score from the evaluation object using the results from testing data transformation
evaluation.evaluate(res)

Out[35]: 0.7467398481346978

In [0]:
# The AUROC is ~0.75 which is quite decent

In [0]:
# Sneak peek into which customers might actually churn and who wouldn't
res.show()

+----+--------------+---------------+-----+---------+-----+--------------------+--------------------+--------------------+----------+
 Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn| features| rawPrediction| probability|prediction|
+----+--------------+---------------+-----+---------+-----+--------------------+--------------------+--------------------+----------+
28.0| 11128.95| 1| 5.12| 8.0| 0|[28.0,11128.95,1....|[3.80776866181817...|[0.97828438157686...| 0.0|
29.0| 12711.15| 0| 5.74| 7.0| 0|[29.0,12711.15,0....|[5.01075969076212...|[0.99337830175670...| 0.0|
29.0| 13255.05| 1| 4.89| 8.0| 0|[29.0,13255.05,1....|[3.85007061183580...|[0.97916509606875...| 0.0|
30.0| 12788.37| 0| 4.31| 10.0| 0|[30.0,12788.37,0....|[2.29312763539775...|[0.90830627114070...| 0.0|
31.0| 10058.87| 1| 6.54| 7.0| 0|[31.0,10058.87,1....|[4.14893054770928...|[0.98446389490461...| 0.0|
31.0| 10182.6| 1| 3.77| 8.0| 0|[31.0,10182.6,1.0...|[4.32323995062845...|[0.98691658275357...| 0.0|
31.0| 11297.57| 1| 6.23| 10.0| 1|[31.0,11297.57,1....|[0.93819264884991...|[0.71873443638163...| 0.0|
32.0| 6367.22| 1| 2.84| 10.0| 0|[32.0,6367.22,1.0...|[2.52984236012503...|[0.92620757989328...| 0.0|
32.0| 9036.27| 0| 7.14| 11.0| 0|[32.0,9036.27,0.0...|[-0.1786608902000...|[0.45545320786415...| 1.0|
32.0| 9472.72| 1| 1.0| 10.0| 0|[32.0,9472.72,1.0...|[3.35188664205860...|[0.96616656205504...| 0.0|
32.0| 9885.12| 1| 6.92| 9.0| 1|[32.0,9885.12,1.0...|[1.70528595487260...|[0.84622385073949...| 0.0|
32.0| 11540.86| 0| 3.0| 7.0| 0|[32.0,11540.86,0....|[6.17576838891540...|[0.99792510647115...| 0.0|
32.0| 12142.99| 0| 5.01| 7.0| 0|[32.0,12142.99,0....|[5.23341898831651...|[0.99469306094553...| 0.0|
32.0| 12403.6| 0| 4.99| 7.0| 0|[32.0,12403.6,0.0...|[5.23983526604435...|[0.99472682375294...| 0.0|
32.0| 12479.72| 0| 4.67| 8.0| 0|[32.0,12479.72,0....|[4.27418812994239...|[0.98626784943134...| 0.0|
33.0| 4711.89| 0| 4.97| 7.0| 0|[33.0,4711.89,0.0...|[5.29339800609053...|[0.99500046355591...| 0.0|
33.0| 13314.19| 0| 5.45| 9.0| 0|[33.0,13314.19,0....|[2.74690986173267...|[0.93973859299705...| 0.0|
34.0| 5447.16| 1| 4.13| 9.0| 0|[34.0,5447.16,1.0...|[2.97100982723883...|[0.95124713012318...| 0.0|
34.0| 6461.86| 1| 4.61| 8.0| 0|[34.0,6461.86,1.0...|[3.85006386511339...|[0.97916495842970...| 0.0|
34.0| 7818.13| 0| 6.11| 8.0| 0|[34.0,7818.13,0.0...|[3.57331257412726...|[0.97270328107940...| 0.0|
+----+--------------+---------------+-----+---------+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

## Regression on unforeseen data

In [0]:
# Import the new data file
new_cust=spark.read.csv('dbfs:/FileStore/shared_uploads/hrishagni95@gmail.com/new_customers-2.csv',inferSchema=True,header=True)

In [0]:
new_cust.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+

In [0]:
# Use the previously fitted model to transform the new dataset so as to extract the results
new_res=fit_model.transform(new_cust)

In [0]:
# Display the prediction of who might churn
new_res.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company| features| rawPrediction| probability|prediction|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|[37.0,9935.53,1.0...|[2.24496186126577...|[0.90421506875202...| 0.0|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|[23.0,7526.94,1.0...|[-5.6792953964487...|[0.00340433540746...| 1.0|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|[65.0,100.0,1.0,1...|[-3.4709984298370...|[0.03014877387259...| 1.0|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|[32.0,6487.5,0.0,...|[-4.5436885357040...|[0.01052221543599...| 1.0|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|[32.0,13147.71,1....|[1.34906866697415...|[0.79397732493732...| 0.0|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|[22.0,8445.26,1.0...|[-1.8253390743619...|[0.13879445883582...| 1.0|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+--------------------+--------------------+--------------------+----------+